In [3]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

# **Extracting features of 2023 movies from Wikipedia**

In [4]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2023"

In [5]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [6]:
tables = soup.find_all('table',class_='wikitable sortable')


In [7]:
len(tables)

4

In [8]:
type(tables[0])

bs4.element.Tag

In [9]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [10]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)


<ipython-input-10-29c6c5e572fc>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)


In [11]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,6,M3GAN,Universal Pictures / Blumhouse Productions / A...,Gerard Johnstone (director); Akela Cooper (scr...,[3]
1,J A N U A R Y,6,The Old Way,Saban Films / Saturn Films,Brett Donowho (director); Carl W. Lucas (scree...,[4]
2,J A N U A R Y,11,The Devil Conspiracy,Samuel Goldwyn Films,Nathan Frankowski (director); Ed Alan (screenp...,[5]
3,J A N U A R Y,13,Plane,Lionsgate / MadRiver Pictures / Di Bonaventura...,Jean-François Richet (director); Charles Cummi...,[6]
4,J A N U A R Y,13,House Party,Warner Bros. Pictures / New Line Cinema,"Calmatic (director); Jamal Olori, Stephen Glov...",[7]
...,...,...,...,...,...,...
221,D E C E M B E R,20,Untitled Ghostbusters: Afterlife sequel,Columbia Pictures / Bron Creative / Ghost Corps,Gil Kenan (director/screenplay); Jason Reitman...,[198]
222,D E C E M B E R,22,Rebel Moon,Netflix / The Stone Quarry,Zack Snyder (director); Shay Hatten (screenpla...,[96]
223,D E C E M B E R,22,Migration,Universal Pictures / Illumination,Benjamin Renner (director); Mike White (screen...,[199]
224,D E C E M B E R,25,The Color Purple,Warner Bros. Pictures / Harpo Films / Amblin E...,"Blitz Bazawule (director), Marcus Gardley (scr...",[197]


In [12]:
df_2023 = df[['Title','Cast and crew']]

In [13]:
df_2023

,Title,Cast and crew
0,M3GAN,Gerard Johnstone (director); Akela Cooper (scr...
1,The Old Way,Brett Donowho (director); Carl W. Lucas (scree...
2,The Devil Conspiracy,Nathan Frankowski (director); Ed Alan (screenp...
3,Plane,Jean-François Richet (director); Charles Cummi...
4,House Party,"Calmatic (director); Jamal Olori, Stephen Glov..."
...,...,...
221,Untitled Ghostbusters: Afterlife sequel,Gil Kenan (director/screenplay); Jason Reitman...
222,Rebel Moon,Zack Snyder (director); Shay Hatten (screenpla...
223,Migration,Benjamin Renner (director); Mike White (screen...
224,The Color Purple,"Blitz Bazawule (director), Marcus Gardley (scr..."


In [14]:
!pip install tmdbv3api


In [15]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '9aa119e1665b53fac6c324512f48325e'

In [16]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " "
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [17]:
df_2023['genres'] = df_2023['Title'].map(lambda x: get_genre(str(x)))

<ipython-input-17-2ed0b991c088>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023['genres'] = df_2023['Title'].map(lambda x: get_genre(str(x)))


In [18]:
df_2023

,Title,Cast and crew,genres
0,M3GAN,Gerard Johnstone (director); Akela Cooper (scr...,Science Fiction Horror
1,The Old Way,Brett Donowho (director); Carl W. Lucas (scree...,Western
2,The Devil Conspiracy,Nathan Frankowski (director); Ed Alan (screenp...,Horror Fantasy Science Fiction Thriller
3,Plane,Jean-François Richet (director); Charles Cummi...,Action Adventure Thriller
4,House Party,"Calmatic (director); Jamal Olori, Stephen Glov...",Comedy
...,...,...,...
221,Untitled Ghostbusters: Afterlife sequel,Gil Kenan (director/screenplay); Jason Reitman...,Fantasy Adventure Comedy
222,Rebel Moon,Zack Snyder (director); Shay Hatten (screenpla...,Science Fiction Fantasy Action
223,Migration,Benjamin Renner (director); Mike White (screen...,Animation Comedy Adventure
224,The Color Purple,"Blitz Bazawule (director), Marcus Gardley (scr...",Drama


In [19]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [20]:
df_2023['director_name'] = df_2023['Cast and crew'].map(lambda x: get_director(str(x)))


<ipython-input-20-6179b260bd79>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023['director_name'] = df_2023['Cast and crew'].map(lambda x: get_director(str(x)))


In [21]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])


In [22]:
df_2023['actor_1_name'] = df_2023['Cast and crew'].map(lambda x: get_actor1(str(x)))

<ipython-input-22-07af9e2bc7c7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023['actor_1_name'] = df_2023['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [23]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])


In [24]:
df_2023['actor_2_name'] = df_2023['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [25]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [26]:
df_2023['actor_3_name'] = df_2023['Cast and crew'].map(lambda x: get_actor3(str(x)))
df_2023


,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,M3GAN,Gerard Johnstone (director); Akela Cooper (scr...,Science Fiction Horror,Gerard Johnstone,Allison Williams,Violet McGraw,Amie Donald
1,The Old Way,Brett Donowho (director); Carl W. Lucas (scree...,Western,Brett Donowho,Nicolas Cage,Ryan Kiera Armstrong,NaN
2,The Devil Conspiracy,Nathan Frankowski (director); Ed Alan (screenp...,Horror Fantasy Science Fiction Thriller,Nathan Frankowski,Alice Orr-Ewing,Joe Doyle,Eveline Hall
3,Plane,Jean-François Richet (director); Charles Cummi...,Action Adventure Thriller,Jean-François Richet,Gerard Butler,Mike Colter,Yoson An
4,House Party,"Calmatic (director); Jamal Olori, Stephen Glov...",Comedy,Calmatic,Tosin Cole,Jacob Latimore,Karen Obilom
...,...,...,...,...,...,...,...
221,Untitled Ghostbusters: Afterlife sequel,Gil Kenan (director/screenplay); Jason Reitman...,Fantasy Adventure Comedy,Gil Kenan,Mckenna Grace,Carrie Coon,Finn Wolfhard
222,Rebel Moon,Zack Snyder (director); Shay Hatten (screenpla...,Science Fiction Fantasy Action,Zack Snyder,Sofia Boutella,Charlie Hunnam,Ray Fisher
223,Migration,Benjamin Renner (director); Mike White (screen...,Animation Comedy Adventure,Benjamin Renner,Kumail Nanjiani,Elizabeth Banks,Keegan-Michael Key
224,The Color Purple,"Blitz Bazawule (director), Marcus Gardley (scr...",Drama,Blitz Bazawule,Fantasia Barrino,Taraji P. Henson,Danielle Brooks


In [27]:
df_2023 = df_2023.rename(columns={'Title':'movie_title'})

In [28]:
new_df23 = df_2023.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]
new_df23


,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Gerard Johnstone,Allison Williams,Violet McGraw,Amie Donald,Science Fiction Horror,M3GAN
1,Brett Donowho,Nicolas Cage,Ryan Kiera Armstrong,NaN,Western,The Old Way
2,Nathan Frankowski,Alice Orr-Ewing,Joe Doyle,Eveline Hall,Horror Fantasy Science Fiction Thriller,The Devil Conspiracy
3,Jean-François Richet,Gerard Butler,Mike Colter,Yoson An,Action Adventure Thriller,Plane
4,Calmatic,Tosin Cole,Jacob Latimore,Karen Obilom,Comedy,House Party
...,...,...,...,...,...,...
221,Gil Kenan,Mckenna Grace,Carrie Coon,Finn Wolfhard,Fantasy Adventure Comedy,Untitled Ghostbusters: Afterlife sequel
222,Zack Snyder,Sofia Boutella,Charlie Hunnam,Ray Fisher,Science Fiction Fantasy Action,Rebel Moon
223,Benjamin Renner,Kumail Nanjiani,Elizabeth Banks,Keegan-Michael Key,Animation Comedy Adventure,Migration
224,Blitz Bazawule,Fantasia Barrino,Taraji P. Henson,Danielle Brooks,Drama,The Color Purple


In [29]:
new_df23['comb'] = new_df23['actor_1_name'] + ' ' + new_df23['actor_2_name'] + ' '+ new_df23['actor_3_name'] + ' '+ new_df23['director_name'] +' ' + new_df23['genres']

In [30]:
new_df23.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      1
actor_3_name     12
genres            1
movie_title       0
comb             13
dtype: int64

In [31]:
new_df23 = new_df23.dropna(how='any')

In [32]:
new_df23.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [33]:
new_df23['movie_title'] = new_df23['movie_title'].str.lower()

<ipython-input-33-84b4bf0117a3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df23['movie_title'] = new_df23['movie_title'].str.lower()


In [34]:
new_df23

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Gerard Johnstone,Allison Williams,Violet McGraw,Amie Donald,Science Fiction Horror,m3gan,Allison Williams Violet McGraw Amie Donald Ger...
2,Nathan Frankowski,Alice Orr-Ewing,Joe Doyle,Eveline Hall,Horror Fantasy Science Fiction Thriller,the devil conspiracy,Alice Orr-Ewing Joe Doyle Eveline Hall Nathan ...
3,Jean-François Richet,Gerard Butler,Mike Colter,Yoson An,Action Adventure Thriller,plane,Gerard Butler Mike Colter Yoson An Jean-Franço...
4,Calmatic,Tosin Cole,Jacob Latimore,Karen Obilom,Comedy,house party,Tosin Cole Jacob Latimore Karen Obilom Calmati...
5,John Hyams,Gideon Adlon,Bethlehem Million,Marc Menchaca,Horror Thriller,sick,Gideon Adlon Bethlehem Million Marc Menchaca J...
...,...,...,...,...,...,...,...
221,Gil Kenan,Mckenna Grace,Carrie Coon,Finn Wolfhard,Fantasy Adventure Comedy,untitled ghostbusters: afterlife sequel,Mckenna Grace Carrie Coon Finn Wolfhard Gil Ke...
222,Zack Snyder,Sofia Boutella,Charlie Hunnam,Ray Fisher,Science Fiction Fantasy Action,rebel moon,Sofia Boutella Charlie Hunnam Ray Fisher Zack ...
223,Benjamin Renner,Kumail Nanjiani,Elizabeth Banks,Keegan-Michael Key,Animation Comedy Adventure,migration,Kumail Nanjiani Elizabeth Banks Keegan-Michael...
224,Blitz Bazawule,Fantasia Barrino,Taraji P. Henson,Danielle Brooks,Drama,the color purple,Fantasia Barrino Taraji P. Henson Danielle Bro...


In [35]:
old_df = pd.read_csv('onemore_data.csv')

In [36]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6750,Kasi Lemmons,Naomi Ackie,Stanley Tucci,Ashton Sanders,Music History Drama,whitney houston: i wanna dance with somebody,Naomi Ackie Stanley Tucci Ashton Sanders Kasi ...
6751,Scott Cooper,Christian Bale,Harry Melling,Gillian Anderson,Crime,the pale blue eye,Christian Bale Harry Melling Gillian Anderson ...
6752,Sarah Polley,Rooney Mara,Claire Foy,Jessie Buckley,Drama,women talking,Rooney Mara Claire Foy Jessie Buckley Sarah Po...
6753,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,Comedy Drama,a man called otto,Tom Hanks Mariana Treviño Rachel Keller Marc F...


In [37]:
final_df = old_df.append(new_df23,ignore_index=True)

<ipython-input-37-df448504f459>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = old_df.append(new_df23,ignore_index=True)


In [38]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6963,Gil Kenan,Mckenna Grace,Carrie Coon,Finn Wolfhard,Fantasy Adventure Comedy,untitled ghostbusters: afterlife sequel,Mckenna Grace Carrie Coon Finn Wolfhard Gil Ke...
6964,Zack Snyder,Sofia Boutella,Charlie Hunnam,Ray Fisher,Science Fiction Fantasy Action,rebel moon,Sofia Boutella Charlie Hunnam Ray Fisher Zack ...
6965,Benjamin Renner,Kumail Nanjiani,Elizabeth Banks,Keegan-Michael Key,Animation Comedy Adventure,migration,Kumail Nanjiani Elizabeth Banks Keegan-Michael...
6966,Blitz Bazawule,Fantasia Barrino,Taraji P. Henson,Danielle Brooks,Drama,the color purple,Fantasia Barrino Taraji P. Henson Danielle Bro...


In [39]:
final_df.to_csv('main_data.csv',index=False)

In [46]:
import pickle

In [47]:
df_new = pd.read_csv('main_data.csv')

In [49]:
df_new.to_pickle('movies.pkl')

In [51]:
pickle.dump(df_new, open('movies.pkl','wb'))

In [52]:
df_new.to_dict()

{'director_name': {0: 'James Cameron',
  1: 'Gore Verbinski',
  2: 'Sam Mendes',
  3: 'Christopher Nolan',
  4: 'Doug Walker',
  5: 'Andrew Stanton',
  6: 'Nathan Greno',
  7: 'Joss Whedon',
  8: 'David Yates',
  9: 'Zack Snyder',
  10: 'Bryan Singer',
  11: 'Marc Forster',
  12: 'Gore Verbinski',
  13: 'Gore Verbinski',
  14: 'Zack Snyder',
  15: 'Andrew Adamson',
  16: 'Rob Marshall',
  17: 'Barry Sonnenfeld',
  18: 'Peter Jackson',
  19: 'Marc Webb',
  20: 'Ridley Scott',
  21: 'Peter Jackson',
  22: 'Chris Weitz',
  23: 'James Cameron',
  24: 'Anthony Russo',
  25: 'Peter Berg',
  26: 'Colin Trevorrow',
  27: 'Sam Raimi',
  28: 'Shane Black',
  29: 'Brett Ratner',
  30: 'Dan Scanlon',
  31: 'Michael Bay',
  32: 'Michael Bay',
  33: 'Marc Webb',
  34: 'John Lasseter',
  35: 'Martin Campbell',
  36: 'Lee Unkrich',
  37: 'McG',
  38: 'James Wan',
  39: 'Marc Forster',
  40: 'Bryan Singer',
  41: 'J.J. Abrams',
  42: 'Bryan Singer',
  43: 'Mike Newell',
  44: 'Guillermo del Toro',
  45

In [54]:
pickle.dump(df_new.to_dict(), open('movies_dict.pkl','wb'))